In [1]:
from numpy import dot
from numpy.linalg import norm
import openai
import copy
import random
random.seed(4)

In [3]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
"""
def get_response(prompt=None, content=None):
    openai.api_type = "azure"
    openai.api_base = "https://bigo-chatgpt-9.openai.azure.com/"
    openai.api_version = "2023-03-15-preview"
    # key1: 19ea901e8e10475da1bb0537abf8e5a4
    # key2: 548e5c0c2aff453e932948927a27bde6
    openai.api_key = "19ea901e8e10475da1bb0537abf8e5a4"

    # role : system|user|assistant
    gpt_config = {'engine': 'gpt-35-turbo',
                'role': 'user',
                }
    message_list = [
            {"role": "system",
            "content": f"{content}"},
            {"role": "user", "content": f"{prompt}"}
            ]
    response = openai.ChatCompletion.create(
        engine=gpt_config['engine'],
        temperature=0.6,
        messages=message_list
    )
    return response['choices'][0]['message']['content']
"""

def get_response(prompt=None, content=None):
    key = 'bca8eef9f9c04c7bb1e573b4353e71ae'

    openai.api_type = "azure"
    openai.api_base = "https://gpt4-test-cj-0803.openai.azure.com/"
    openai.api_version = "2023-03-15-preview"
    openai.api_key = 'bca8eef9f9c04c7bb1e573b4353e71ae'

    message_list = [
            {"role": "system",
            "content": f"{content}"},
            {"role": "user", "content": f"{prompt}"}
            ]
    response = openai.ChatCompletion.create(
        engine="gpt4-16k",
        messages=message_list,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
    return response['choices'][0]['message']['content']

### 定义embedding获取函数
def get_embedding(text, model="text-embedding-ada-002"):
    openai_api_key = "548e5c0c2aff453e932948927a27bde6"
    openai.api_key = openai_api_key
    openai.api_type = "azure"
    # openai.api_version = "2023-06-15-preview"
    openai.api_version = "2023-03-15-preview"
    openai.api_base = "https://bigo-chatgpt-9.openai.azure.com/"
    res = openai.Embedding.create(
    input=[text], deployment_id="text-embedding-ada-002")['data'][0]['embedding']
    return res

## 定义场景

In [17]:
### 固定一个room的场景，所有的perceived events都是事先定义好的
perceived_evn_Jhon = ["房间里的烤箱中正在炙烤面包",
                      "洗碗槽里的碗还没有洗碗",
                      "厨房正在烧开水，可以发现水已经快开了",
                      "门口有一些垃圾，该打扫卫生了",
                      "门口的两棵树木中间有秋千可以玩耍",
                      "你的儿子安迪正在向你走来"]
perceived_evn_Andi = ["房间里的烤箱中正在炙烤面包",
                      "洗碗槽里的碗还没有洗碗",
                      "厨房正在烧开水，可以发现水已经快开了",
                      "门口有一些垃圾，该打扫卫生了",
                      "门口的两棵树木中间有秋千可以玩耍",
                      "你的父亲乔恩正站在门口"]

## 定义人物

In [7]:
### 定义persona大类
class Persona():
    def __init__(self, name, persona, skills, interests, identity, associative_memory=None):
        self.name = name
        self.persiona = persona
        self.skills = skills
        self.interests = interests
        self.identity = identity
        self.asso_mem = associative_memory if associative_memory is not None else None

    def get_basic_info_prompt(self):
        prompt_template = (
            "你是沙盒世界的一员，你的名字是{name}。\n"
            "你现在正处于一个家庭中。你的身份是：\n"
            "{identity}\n\n"
            "以下是你的性格:\n"
            "{persona}\n\n"
            "你掌握的技巧:\n"
            "{skills}\n\n"
            "你的兴趣爱好: {interests}\n"
        )
        return prompt_template.format_map({'name':self.name, 'persona':self.persiona, 'skills':self.skills, 'identity':self.identity, 'interests':self.interests})
    
    def get_summarized_asso_mem(self):
        if self.asso_mem is None or self.asso_mem is []:
            print("还没有历史记忆")
            return 
        else:
            events_prompt = ''
            for i in range(len(self.asso_mem)):
                event = self.asso_mem[i]
                events_prompt = events_prompt + f'事件: {event}'
            summarize_prompt = '请总结这些过往的事件:' + events_prompt + '每个事件用一句话去总结'
            #print(summarize_prompt)
            summarized_asso_mem = get_response(summarize_prompt)
            summarized_events = [eve for eve in summarized_asso_mem.split('\n') if eve != ""]
            return summarized_events

In [18]:
### 现在初始化几个人格
jhon_dic = {'name': "乔恩",
                'persona': "温和友善： 乔恩是一个亲切友好的人，总是愿意帮助他人，因此在社交圈中备受喜爱。\n勤奋和有责任感： 他非常注重工作，总是尽最大努力完成任务。他喜欢有组织和计划的生活，这使他在工作和生活中都表现出色。\n耐心： 乔恩拥有出色的耐心，不易冲动，能够冷静地应对压力和困难。\n好奇心旺盛： 他对世界充满好奇，喜欢学习新事物，经常探索新的兴趣领域。",
                'skills': "沟通技巧： 乔恩是一位出色的沟通者，擅长倾听他人并表达自己的观点，这使他在职业和社交场合都表现出色。\n烹饪： 他对烹饪非常有天赋，经常烹制美味的餐点，也喜欢尝试各种各样的烹饪风格。\n音乐才能： 乔恩会弹奏吉他，享受音乐的世界，也喜欢在朋友和家人聚会时演奏音乐。\n编程技能： 他拥有出色的计算机编程技能，是一位优秀的软件工程师。",
                'identity':"安迪的父亲。乔恩是一位退休的军队上校，他曾在军队中担任高级情报官员，如今过着宁静的退休生活。",
                'interests': "徒步旅行： 乔恩热爱大自然，经常进行徒步旅行，探索山川河流，欣赏自然之美。\n阅读： 他是一位书虫，热衷于各种类型的书籍，特别是科幻小说和历史书籍。\n篮球：乔恩喜欢打篮球，经常和朋友一起在周末参加篮球比赛。"}
jhon = Persona(name=jhon_dic['name'], persona=jhon_dic['persona'], skills=jhon_dic['skills'], identity=jhon_dic['identity'] ,interests=jhon_dic['interests'])


andi_dic= {'name': "安迪",
                'persona': "自信和坚定： 安迪有坚定的信念，自信而果断，能够在压力下保持冷静。\n创造力： 他是一位富有创造力的思考者，常常能够提出新颖的想法和解决问题的方法。\n友善和关心他人： 安迪非常关心他人，善待朋友和家人，经常主动帮助别人，建立了强大的社交网络。\n冒险精神： 他喜欢尝试新事物，勇于冒险，经常参加各种刺激的户外活动。",
                'identity': "乔恩的儿子。安迪是一名年轻的环保活动家，他致力于保护自然环境，参与各种社会和环保项目，以推动可持续发展和环境保护。",
                'skills': "绘画和艺术： 安迪是一位出色的艺术家，擅长绘画和创作艺术作品，他的作品经常展示在当地画廊。\n瑜伽： 他是一名有经验的瑜伽修行者，经常进行冥想和身体训练，保持身心健康。\n写作： 安迪热爱写作，擅长创作诗歌和散文，也有一些出版的短篇小说。\n户外冒险： 他喜欢登山、潜水和徒步旅行，经常在自然中寻找刺激和冒险。",
                'interests': "探索古迹： 安迪热爱历史和文化，经常前往古迹和博物馆，探索人类历史的奥秘。\n音乐： 他是一位音乐爱好者，喜欢演奏吉他，也热衷于参加音乐节和音乐会。\n烹饪： 安迪对烹饪有浓厚兴趣，经常烹制各种美食，特别喜欢尝试国际美食。"}
andi = Persona(name=andi_dic['name'], persona=andi_dic['persona'], identity=andi_dic['identity'] ,skills=andi_dic['skills'] ,interests=andi_dic['interests'])

## 定义过往记忆

In [19]:
pre_mem = ["当乔恩逛街时，他在街头巧遇一位老友，促成了意外的聚会。",
           "安迪在学校里参加了一场足球比赛，打进了关键进球，让他的团队获胜。",
           "乔恩和安迪一起去了野外露营，他们一起生起了篝火，分享了各自最喜欢的故事。",
           "一天，乔恩和安迪一起去了当地的动物收容所，他们领养了一只可爱的小猫，给它取名“橘子”。",
           "安迪在学校的科学比赛中获得了第一名，让乔恩为他感到骄傲。",
           "乔恩带着安迪去了一个农场，他们一起喂养了农场里的动物，体验了农村生活。",
           "安迪在家里为乔恩准备了一份惊喜早餐，庆祝父亲的生日。",
           "乔恩和安迪一起参加了一个社区义工活动，帮助清理公园并结识了新朋友。",
           "家里的电视坏了，于是乔恩和安迪一起决定玩一场激烈的家庭桌游比赛，度过了一个欢乐的周末。",
           "一天，乔恩和安迪一起去了当地的冰淇淋店，他们品尝了各种口味的冰淇淋，互相分享自己的最爱。",
           "安迪在学校的才艺秀上表演了一首吉他独奏，乔恩坐在观众席为他鼓掌。",
           "乔恩和安迪一起去了野外露营，他们在帐篷里度过了一个星光闪烁的夜晚，远离城市的喧嚣。",
           "家里的花园需要整理，乔恩和安迪一起劳动，植树、修剪花草，让花园焕发生机。",
           "一天，乔恩带着安迪去参观了博物馆，他们一起探索历史和艺术的奇妙世界。",
           "安迪带了一只迷路的小猫回家，他们决定一起照顾它，给它取名“迷失”。",
           "乔恩和安迪一同制作了一顿美味的晚餐，学着做新的菜肴，度过了一个美食的夜晚。",
           "家里的书架需要整理，乔恩和安迪一起整理并分享了彼此最喜欢的书籍。",
           "乔恩和安迪一同去了动物园，看到了一只可爱的大熊猫，拍下了许多照片。",
           "家里停电了，乔恩和安迪点起蜡烛，一起讲了很多幽默的故事，渡过了一个温馨的夜晚。",]

## Retrieve

In [10]:
### 定义匹配函数，对于每个当下接收到的信息 perceived，将perceived每个事件的embedding与过往事件的embedding做向量匹配
### 匹配方式是余弦相似度，每个perceived事件返回最高的两个过往事件
def get_revelant_past_mem(perceived, prev_mem, top_k=2):
    sim_score = [[0 for _ in range(len(prev_mem))] for _ in range(len(perceived))]
    relevent_idx = {}
    perceived_emb = [get_embedding(perceived[i]) for i in range(len(perceived))]
    prev_mem_emb = [get_embedding(prev_mem[i]) for i in range(len(prev_mem))]
    for i in range(len(perceived)):
        for j in range(len(prev_mem)):
            sim_score[i][j] = cos_sim(perceived_emb[i], prev_mem_emb[j])
        relevent_idx[i], _ = get_top_k_idx(sim_score[i])
    return relevent_idx, perceived_emb, prev_mem_emb

def cos_sim(a, b):
  ## 返回一个值 0-1 
  return dot(a, b)/(norm(a)*norm(b))


def get_top_k_idx(scores, top_k=2):
    t = copy.deepcopy(scores)
    # 求m个最大的数值及其索引
    max_number = []
    max_index = []
    for _ in range(top_k):
        number = max(t)
        index = t.index(number)
        t[index] = 0
        max_number.append(number)
        max_index.append(index)
    return max_index, max_number

print(get_revelant_past_mem(perceived=perceived_evn_Jhon , prev_mem=pre_mem))

({0: [15, 6], 1: [15, 8], 2: [6, 18], 3: [12, 7], 4: [12, 2], 5: [6, 1]}, [[-0.007456143386662006, -0.005820966325700283, -0.0004908848786726594, 0.0009278719662688673, -0.024225832894444466, -0.016438011080026627, -0.02557908371090889, -0.005708195269107819, -0.009565621614456177, -0.012769640423357487, 0.00339307589456439, 0.020351821556687355, -0.013890715315937996, -0.004411330446600914, -0.005681660957634449, 0.004918799269944429, 0.031602371484041214, 0.016729887574911118, 0.015442973002791405, -0.025035129860043526, -0.006212347187101841, -0.013758043758571148, 0.005814332515001297, 0.003169192699715495, -0.024849388748407364, 0.011356688104569912, 0.0013408123049885035, -0.010355018079280853, -0.0031841183081269264, 0.01744631491601467, 0.04237530753016472, 0.009333446621894836, -0.0304348636418581, -0.017127903178334236, -0.013446265831589699, -0.025738289579749107, -0.01227212231606245, 0.00481929536908865, 0.009479384869337082, -0.007250502705574036, 0.012723205611109734, -0

In [11]:
def get_focused_env(perceived, summarized_prev_mem):
    reterived_mem_dic, _, _ = get_revelant_past_mem(perceived=perceived, prev_mem=summarized_prev_mem)
    reterived_mem_idx = []
    for k in reterived_mem_dic.keys():
        for i in range(len(reterived_mem_dic[k])):
            if reterived_mem_dic[k][i] not in reterived_mem_idx:
                reterived_mem_idx.append(reterived_mem_dic[k][i])
    focused_event = [summarized_prev_mem[reterived_mem_idx[i]] for i in range(len(reterived_mem_idx))]
    return focused_event

In [12]:
get_focused_env(perceived_evn_Jhon, pre_mem)

['乔恩和安迪一同制作了一顿美味的晚餐，学着做新的菜肴，度过了一个美食的夜晚。',
 '安迪在家里为乔恩准备了一份惊喜早餐，庆祝父亲的生日。',
 '家里的电视坏了，于是乔恩和安迪一起决定玩一场激烈的家庭桌游比赛，度过了一个欢乐的周末。',
 '家里停电了，乔恩和安迪点起蜡烛，一起讲了很多幽默的故事，渡过了一个温馨的夜晚。',
 '家里的花园需要整理，乔恩和安迪一起劳动，植树、修剪花草，让花园焕发生机。',
 '乔恩和安迪一起参加了一个社区义工活动，帮助清理公园并结识了新朋友。',
 '乔恩和安迪一起去了野外露营，他们一起生起了篝火，分享了各自最喜欢的故事。',
 '安迪在学校里参加了一场足球比赛，打进了关键进球，让他的团队获胜。']

In [20]:
reterived_mem_dic_Jhon, _, _ = get_revelant_past_mem(perceived=perceived_evn_Jhon, prev_mem=pre_mem)
reterived_mem_dic_Andi, _, _ = get_revelant_past_mem(perceived=perceived_evn_Andi, prev_mem=pre_mem)
reterived_mem_idx_Jhon = []
reterived_mem_idx_Andi = []
for k in reterived_mem_dic_Jhon.keys():
    for i in range(len(reterived_mem_dic_Jhon[k])):
        if reterived_mem_dic_Jhon[k][i] not in reterived_mem_idx_Jhon:
            reterived_mem_idx_Jhon.append(reterived_mem_dic_Jhon[k][i])
for k in reterived_mem_dic_Andi.keys():
    for i in range(len(reterived_mem_dic_Andi[k])):
        if reterived_mem_dic_Andi[k][i] not in reterived_mem_idx_Andi:
            reterived_mem_idx_Andi.append(reterived_mem_dic_Andi[k][i])
reterived_mem_idx_Jhon

[15, 6, 8, 18, 12, 7, 2, 1]

In [21]:
focused_event_Jhon = [pre_mem[reterived_mem_idx_Jhon[i]] for i in range(len(reterived_mem_idx_Jhon))]
focused_event_Andi =  [pre_mem[reterived_mem_idx_Andi[i]] for i in range(len(reterived_mem_idx_Andi))]

## Plan

In [22]:
current_jhon = "乔恩正站在门口发呆"
current_andi = "安迪正在院子里玩耍"
react_prompt_jhon = ("{person_info}\n"
                "根据你现在的状态：\n"
                "{current}"
                "以及当前接收到的状况：\n"
                "{perceived}"
                "跟当下相关的过往记忆是：\n"
                "{focused_event}"
                "做出是否跟你儿子对话的决定、n"
                "如果是返回：1，如果不是返回：0。")

react_prompt_andi = ("{person_info}\n"
                "根据你现在的状态：\n"
                "{current}"
                "以及当前接收到的状况：\n"
                "{perceived}"
                "跟当下相关的过往记忆是：\n"
                "{focused_event}"
                "做出是否跟你父亲对话的决定、n"
                "如果是返回：1，如果不是返回：0")

chat_prompt = ("请以安迪和乔恩的身份展开一段对话\n"
               "乔恩的人物信息\n"
               "{info_j}\n"
               "安迪的人物信息\n"
               "{info_a}\n"
               "乔恩的人物状态\n"
               "{state_j}\n"
               "安迪的人物状态\n"
               "{state_a}\n"
               )

plan_prompt = (
            "你是沙盒世界的一员，你的信息是{person_info}。\n"
            "你现在正在面对一些场景\n"
            "{perceived}\n"
            "你现在的状态是\n"
            "{current}\n"
            "跟当下相关的过往记忆是：\n"
            "{focused_event}"
            "现在请根据这些经历，制定你的下一步计划\n"
        )

state_prompt = (
            "你是沙盒世界的一员，你的信息是{person_info}。\n"
            "你现在的计划是\n"
            "{plan}\n"
            "请根据你的计划生成你现在的状态\n"
        )

get_summarized_chat = (
            "请总结以下的对话内容：\n"
            "{chat}"
)

reflection_prompt = (
    "请总结过往发生过的内容并形成一定思考：\n"
            "{prev_mem}"
)

In [23]:
for i in range(5):
    print(f"+++++++++++++++++++      轮数{i}   +++++++++++++++++++++++++++++++")
    react_prompt_j = react_prompt_jhon.format_map({'person_info':jhon.get_basic_info_prompt(), 'current':current_jhon, 'perceived':perceived_evn_Jhon, 'focused_event':focused_event_Jhon})
    react_prompt_a = react_prompt_jhon.format_map({'person_info':andi.get_basic_info_prompt(), 'current':current_andi, 'perceived':perceived_evn_Andi, 'focused_event':focused_event_Andi})
    res_j = get_response(react_prompt_j)
    res_a = get_response(react_prompt_a)
    print(res_j, res_a)
    if '1' in res_j and '1' in res_a:
        chat_p = chat_prompt.format_map({'info_j':jhon.get_basic_info_prompt() , 'info_a':andi.get_basic_info_prompt() , 'state_j':current_jhon , 'state_a': 'current_andi'})
        chat_content = get_response(chat_p)
        print("+++++++++++++++++++      开始聊天    +++++++++++++++++++++++++++++++")
        print(chat_content)
        summarized_chat = get_response(get_summarized_chat.format_map({'chat':chat_content}))
        perceived_evn_Jhon.append(summarized_chat)
        perceived_evn_Andi.append(summarized_chat)
        current_jhon = "刚刚结束一段关于：\n" + summarized_chat + "\n 的对话"
        current_andi = "刚刚结束一段关于：\n" + summarized_chat + "\n 的对话"
        focused_event_Jhon = get_focused_env(perceived_evn_Jhon, pre_mem)
        focused_event_Andi = get_focused_env(perceived_evn_Andi, pre_mem)
    else:
        plan_j = plan_prompt.format_map({'person_info':jhon.get_basic_info_prompt(), 'current':current_jhon, 'perceived':perceived_evn_Jhon, 'focused_event':focused_event_Jhon})
        plan_a = plan_prompt.format_map({'person_info':andi.get_basic_info_prompt(), 'current':current_andi, 'perceived':perceived_evn_Andi, 'focused_event':focused_event_Andi})
        print("+++++++++++++++++++      父子制定的计划    +++++++++++++++++++++++++++++++")
        print(plan_j)
        print(plan_a)
        current_jhon = get_response(state_prompt.format_map({'person_info':jhon.get_basic_info_prompt(), 'plan':plan_j}))
        current_andi = get_response(state_prompt.format_map({'person_info':andi.get_basic_info_prompt(), 'plan':plan_a}))
        print("+++++++++++++++++++      父子现在的状态    +++++++++++++++++++++++++++++++")
        print(current_jhon)
        print(current_andi)
        perceived_evn_Jhon.append(plan_j)
        perceived_evn_Andi.append(plan_a)
        focused_event_Jhon = get_focused_env(perceived_evn_Jhon, pre_mem)
        focused_event_Andi = get_focused_env(perceived_evn_Andi, pre_mem)

    if i != 0 and (i-1) % 5 == 0:
        reflection_pro_j = get_response(reflection_prompt.format_map({'prev_mem':perceived_evn_Jhon[:-5]}))
        reflection_pro_a = get_response(reflection_prompt.format_map({'prev_mem':perceived_evn_Andi[:-5]}))
        print("+++++++++++++++++++      父子当下的反思    +++++++++++++++++++++++++++++++")
        print(reflection_pro_j)
        print(reflection_pro_a)
        perceived_evn_Jhon.append(reflection_pro_j)
        perceived_evn_Andi.append(reflection_pro_a)

+++++++++++++++++++      轮数0   +++++++++++++++++++++++++++++++
1 1
+++++++++++++++++++      开始聊天    +++++++++++++++++++++++++++++++
安迪正在家中的书房里写作

乔恩：（走进书房）儿子，你在这儿啊。我刚在门口发呆，你在忙什么呢？
 
安迪：（看起来很专注）爸爸，你来啦。我正在写一篇关于环保的文章，想发表在我们的环保社区。

乔恩：哦，那真是太棒了。你总是如此关心环境问题，我为你感到骄傲。如果需要任何帮助，尽管告诉我。

安迪：谢谢你，爸爸。我知道你以前是个程序员，我想在文章中加入一些数据分析，你能帮我看看这部分吗？

乔恩：当然可以，我很乐意帮忙。另外，你能教我做那个你上次做的意大利面吗？我觉得那个味道太好了，我也想学学。

安迪：哈哈，那当然可以，爸爸。我们可以在晚餐时一起做。
+++++++++++++++++++      轮数1   +++++++++++++++++++++++++++++++
1 1
+++++++++++++++++++      开始聊天    +++++++++++++++++++++++++++++++
乔恩: 安迪，我看了你的文章，你的观点非常有深度，我很喜欢。你需要我帮你分析哪些数据呢？

安迪: 爸爸，我在文章中提到了地球温室气体的排放量，我需要一些具体的数据来支持我的观点。我在网上找到了一些数据，但我不知道如何分析它们。

乔恩: 没问题，安迪。我会帮你分析这些数据。你把数据发送给我，我会尽快给你答复。

安迪: 非常感谢你，爸爸！那我们现在开始做意大利面吗？

乔恩: 是的，我已经迫不及待了。你能提醒我一下我们需要什么食材吗？

安迪: 当然，我们需要意大利面、番茄酱、洋葱、大蒜、橄榄油、牛肉和奶酪。我已经准备好了所有的食材。

乔恩: 很好，那我们开始吧。你先告诉我怎么做，我会跟着你的步骤来。

安迪: 好的，爸爸。让我们开始吧。
+++++++++++++++++++      父子当下的反思    +++++++++++++++++++++++++++++++
过往的内容主要发生在厨房中，包括烤箱正在炙烤面包，洗碗槽里的碗尚未清洗，以及开水已经快烧开。这些情况表明厨房正在进行或准备进行一